In [29]:
from sys import path
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
path.append('../src')
import src

df_temperature = pd.read_csv("../data/temperature-quotidienne-departementale.csv", sep=';')
df_temperature.rename(columns = {"TMin (°C)": "TMin", "TMax (°C)": "TMax", "TMoy (°C)": "TMoy"}, inplace=True)
#We only consider dates before 2023, so dates between 2014 and 2022
df_temperature = df_temperature[df_temperature.Date<"2023"]
df_temperature.head()

,Date,Code INSEE département,Département,TMin,TMax,TMoy
0,2019-10-22,21,Côte-d'Or,10.90,16.00,13.45
1,2019-10-22,73,Savoie,11.70,18.65,15.18
2,2019-10-22,29,Finistère,7.77,15.48,11.63
3,2019-10-22,48,Lozère,9.70,16.50,13.10
4,2019-10-22,49,Maine-et-Loire,4.40,15.10,9.75


In [30]:
# Create a dictionary to map agglomerations to departments
agglomeration_department_map = {
    'Agglomération de Nice-Menton': '06',
    "Agglomération d'Aix-Marseille": '13',
    "Agglomération d'Arles": '13',
    'Agglomération de Besançon': '25',
    'Agglomération de Toulouse': '31',
    'Agglomération de Montpellier': '34',
    'Agglomération de Sète': '34',
    'Agglomération de Rennes': '35',
    'Sud Loire - proche Haute-Loire': '42',
    'Agglomération de Nantes': '44',
    'Agglomération de Lille': '59',
    "Agglomération d'Alençon": '61',
    "Unité urbaine de l'agglomération de Bayonne": '64',
    'Agglomération de Lyon': '69',
    'Agglomération parisienne': '75',
    'Agglomération parisienne (hors Paris)': '77,78,91,92,93,94,95',
    'Paris intra-muros': '75',
    'Agglomération de Toulon': '83',
    'Agglomération de Draguignan': '83',
    'Agglomération de Fréjus': '83',
    'Agglomération de Saint Denis (La Réunion)': '974',
    'Agglomération de La Rochelle': '17',
    'Agglomération de Bordeaux': '33',
    'Agglomération de Teste-de-Buch-Arcachon': '33',
    'Eurométropole de Strasbourg': '67',
    'Agglomération de Marseille': '13',
    'Agglomération de Montbéliard': '25',
    'Agglomération de Brest': '29',
    'Agglomération de Nîmes': '30',
    'Agglomération de Grenoble': '38',
    'Agglomération de Nancy': '54',
    'Agglomération de Douai': '59',
    'Agglomération de Clermont-Ferrand': '63',
    'La Roche-sur-Yon Agglomération': '85',
    "Les Sables d'Olonne Agglomération": '85',
    'Pays de Challans': '85',
    'Agglomération CINOR': '974',
    'Agglomération TCO': '974',
    'Agglomération CIVIS': '974',
    "Agglomération d'Ajaccio": '2A',
    'Golfe du Morbihan - Vannes Agglomération': '56',
    'Lorient Agglomération': '56',
    'Auray Quiberon Terre Atlantique': '56',
    'Agglomération CASUD': '974',
    'Agglomération CIREST': '974',
    'Agglomération de Valence-Romans': '26',
    'Agglomération de Montélimar': '26',
    'Agglomération de Vitré': '35',
    'Agglomération de Tours': '37',
    "Agglomération d'Arras": '62'
}

In [31]:
agglomeration_department_map = {agglomeration:departments.split(",")
                                for agglomeration, departments in agglomeration_department_map.items()}

In [32]:
department_map = dict(df_temperature[["Code INSEE département", "Département"]].drop_duplicates().itertuples(index=False))

In [33]:
agglomeration_department_map = {key:list(map(department_map.get, value))
                                for key, value in agglomeration_department_map.items()}

In [34]:
month_dic1 = {'01':'Jan', '02':'Feb', '03':'Mar', '04':'Apr', '05':'May', '06':'Jun', '07':'Jul', '08':'Aug', '09':'Sep',
             '10':'Oct', '11':'Nov', '12':'Dec'}
month_dic2 = {'Apr': 4, 'Aug': 8, 'Dec': 12, 'Feb': 2, 'Jan': 1, 'Jul': 7, 'Jun': 6, 'Mar': 3, 'May': 5, 'Nov': 11,
              'Oct': 10, 'Sep': 9}
def date_conversion(date):
    date_split = date.split("-")
    return f"{month_dic1[date_split[1]]} {date_split[0]}"

In [35]:
departements_selectionnes = ['29', '36', '83', '54', '30', '31', '56', '13', '61', '25', '59', '69', '34', '44', '06',
                             '35', '974', '75', '42', '43', '64', '77', "95"]
df_temperature = df_temperature[df_temperature['Code INSEE département'].isin(departements_selectionnes)]
df_temperature

,Date,Code INSEE département,Département,TMin,TMax,TMoy
2,2019-10-22,29,Finistère,7.77,15.48,11.63
5,2019-10-23,83,Var,18.56,22.28,20.42
6,2019-10-23,36,Indre,11.40,17.50,14.45
11,2019-10-23,42,Loire,11.75,14.90,13.32
20,2019-10-23,35,Ille-et-Vilaine,9.40,15.35,12.38
...,...,...,...,...,...,...
175274,2022-12-09,34,Hérault,5.93,14.73,10.33
175281,2022-12-10,56,Morbihan,-0.20,6.60,3.20
175283,2022-12-10,64,Pyrénées-Atlantiques,0.83,7.33,4.08
175290,2022-12-11,54,Meurthe-et-Moselle,-3.90,-1.00,-2.45


In [36]:
df_temperature_departments = df_temperature[["Département", "TMin", "TMax", "TMoy"]].groupby("Département")\
.mean().sort_values(by="TMoy").reset_index()
df_temperature_departments.head()

,Département,TMin,TMax,TMoy
0,Haute-Loire,4.560296,15.694962,10.127629
1,Meurthe-et-Moselle,7.026205,16.594688,11.810487
2,Orne,7.514294,16.758434,12.136364
3,Nord,8.916670,15.639814,12.278193
4,Doubs,7.669387,17.241457,12.455422


In [37]:
fig = px.bar(df_temperature_departments, x="TMoy", y="Département", title="Average Temperature by Department",
             labels={"TMoy":"Average Temperature (°C)", "Département":"Department"}, orientation='h')
fig.update_traces(hovertemplate="%{label} : <br> %{value:.2f} °C")
fig.show()

In [38]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_temperature_departments["TMoy"],
                     y=df_temperature_departments["Département"],
                     orientation='h',
                     error_x = {"arrayminus":df_temperature_departments["TMoy"]-df_temperature_departments["TMin"],
                                "array":df_temperature_departments["TMax"]-df_temperature_departments["TMoy"]}
                    )
             )
fig.update_traces(hovertemplate="%{label} : <br> %{value:.2f} °C<extra></extra>")
fig.update_layout(xaxis_title = "Average Temperature (°C)",
                  yaxis_title="Department",
                  title={"text":"Average Temperature by Department", "x":0.5, "xanchor":"center"})
fig.show()

In [39]:
df_temperature["Mois"] = df_temperature["Date"].str.rsplit("-", 1).str[0].apply(date_conversion)
df_temperature["Année"] = df_temperature["Mois"].str.split(" ").str[1]

In [40]:
avg_temperatures = df_temperature.groupby(["Mois", "Année", "Département"]).mean().reset_index()
avg_temperatures_year = {year:avg_temperatures[avg_temperatures["Année"]==year]
                     for year in ("2018", "2019", "2020", "2021", "2022")}

In [41]:
departments = df_temperature_departments["Département"]
data_temperature_evolution = {
    (year, department): temperature[temperature["Département"]==department]\
    .sort_values(by="Mois", key=lambda serie: [month_dic2[e.split()[0]] for e in serie])
    for year, temperature in avg_temperatures_year.items()
    for department in departments
}

In [42]:
charts_temperature_evolution = {
    (year, department): px.bar(data,
                            x="Mois",
                            y="TMoy",
                            labels={"Mois":"Month", "TMoy":"Average Temperature (°C)"},
                            title=f"Evolution of the average temperature in {department} accross months of {year}")
    for (year, department), data in data_temperature_evolution.items()
}

In [43]:
list(charts_temperature_evolution.values())[10]

In [44]:
#This structure is done to avoid a FutureWarning ("The default dtype for empty Series will be 'object' instead of
#'float64' in a future version. Specify a dtype explicitly to silence this warning.")
data_temperature_evolution_agglomeration = {
    key:value.groupby("Mois").mean().reset_index().sort_values(by="Mois",
                                                               key=lambda serie: [month_dic2[e.split()[0]] for e in serie])
    for key, value in [
        ((year, agglomeration), temperature[temperature["Département"].isin(departments)])
        for year, temperature in avg_temperatures_year.items()
        for agglomeration, departments in agglomeration_department_map.items()
    ]
    if len(value)
}

In [45]:
charts_temperature_evolution_agglomeration = {
    (year, agglomeration): px.bar(data,
                            x="Mois",
                            y="TMoy",
                            labels={"Mois":"Month", "TMoy":"Average Temperature (°C)"},
                            title=f"Evolution of the average temperature in {agglomeration} accross months of {year}")
    for (year, agglomeration), data in data_temperature_evolution_agglomeration.items()
}

In [46]:
list(charts_temperature_evolution_agglomeration.values())[50]

In [47]:
list(data_temperature_evolution_agglomeration.values())[50]

,Mois,TMin,TMax,TMoy
4,Jan 2019,3.281935,9.694839,6.487742
3,Feb 2019,4.296071,14.131429,9.213571
7,Mar 2019,6.927419,16.733871,11.830968
0,Apr 2019,9.130667,17.374000,13.251667
8,May 2019,11.778710,20.191613,15.985806
6,Jun 2019,17.118333,27.054000,22.086667
5,Jul 2019,20.894516,30.119677,25.507742
1,Aug 2019,20.068065,29.417419,24.743226
11,Sep 2019,17.045667,25.883000,21.464667
10,Oct 2019,14.096129,21.370645,17.731935


In [48]:
agglomeration_rent_per_year = {2014: {(500, 1000): {'Agglomération de Montpellier', 'Agglomération de Fréjus', 'Agglomération de Toulouse', "Agglomération d'Arles", 'Agglomération de Nantes', "Agglomération d'Aix-Marseille", "Unité urbaine de l'agglomération de Bayonne", 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de Besançon', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Agglomération de Saint Denis (La Réunion)', 'Agglomération de Sète', 'Agglomération parisienne (hors Paris)', 'Agglomération de Lille', 'Agglomération de Draguignan'}, (0, 500): {'Sud Loire - proche Haute-Loire', "Agglomération d'Alençon"}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2015: {(500, 1000): {"Agglomération d'Arles", "Unité urbaine de l'agglomération de Bayonne", 'Eurométropole de Strasbourg', 'Agglomération de Besançon', 'Agglomération de Saint Denis (La Réunion)', 'Agglomération de Sète', 'Agglomération de Draguignan', 'Agglomération de Montpellier', "Agglomération d'Aix-Marseille", 'Agglomération de Lille', 'Agglomération de Teste-de-Buch-Arcachon', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Agglomération de Bordeaux', 'Agglomération de Toulouse', 'Agglomération de Fréjus', 'Agglomération de Nantes', 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de La Rochelle', 'Agglomération parisienne (hors Paris)'}, (0, 500): {'Sud Loire - proche Haute-Loire', "Agglomération d'Alençon"}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2016: {(500, 1000): {'Sud Loire - proche Haute-Loire', "Agglomération d'Arles", "Unité urbaine de l'agglomération de Bayonne", 'Agglomération de Grenoble', 'Eurométropole de Strasbourg', 'Agglomération CIVIS', 'Agglomération de Besançon', 'Agglomération de Sète', 'Agglomération de Draguignan', "Les Sables d'Olonne Agglomération", 'Agglomération de Montpellier', 'Agglomération de Nîmes', 'Agglomération parisienne (hors Paris)', "Agglomération d'Aix-Marseille", 'Agglomération CINOR', 'Agglomération de Lille', 'Pays de Challans', 'Agglomération de Teste-de-Buch-Arcachon', 'Agglomération de Clermont-Ferrand', 'Agglomération TCO', 'Agglomération de Marseille', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Agglomération de Bordeaux', 'Agglomération de Toulouse', 'Agglomération de Douai', 'Agglomération de Fréjus', 'Agglomération de Nantes', 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de La Rochelle', 'Agglomération de Nancy', 'Agglomération de Montbéliard'}, (0, 500): {'La Roche-sur-Yon Agglomération', "Agglomération d'Alençon", 'Agglomération de Brest'}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2017: {(500, 1000): {"Agglomération d'Arles", "Unité urbaine de l'agglomération de Bayonne", 'Agglomération de Grenoble', 'Eurométropole de Strasbourg', 'Agglomération CIVIS', 'Agglomération de Besançon', 'Agglomération de Sète', "Agglomération d'Ajaccio", 'Agglomération de Draguignan', "Les Sables d'Olonne Agglomération", 'Agglomération de Montpellier', 'Agglomération de Nîmes', "Agglomération d'Aix-Marseille", 'Auray Quiberon Terre Atlantique', 'Agglomération CIREST', 'Agglomération CASUD', 'Agglomération CINOR', 'Agglomération de Lille', 'Golfe du Morbihan - Vannes Agglomération', 'Agglomération de Teste-de-Buch-Arcachon', 'Pays de Challans', 'Lorient Agglomération', 'Agglomération de Clermont-Ferrand', 'Agglomération TCO', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Agglomération de Bordeaux', 'Agglomération de Toulouse', 'Agglomération de Fréjus', 'Agglomération de Nantes', 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de La Rochelle', 'Agglomération de Nancy'}, (0, 500): {'Sud Loire - proche Haute-Loire', 'La Roche-sur-Yon Agglomération', "Agglomération d'Alençon", 'Agglomération de Brest'}, (1000, 1500): {'Agglomération parisienne'}}, 2018: {(500, 1000): {"Agglomération d'Arles", 'La Roche-sur-Yon Agglomération', 'Agglomération de Grenoble', "Unité urbaine de l'agglomération de Bayonne", 'Eurométropole de Strasbourg', 'Agglomération CIVIS', 'Agglomération de Besançon', 'Agglomération de Sète', "Agglomération d'Ajaccio", 'Agglomération de Valence-Romans', 'Agglomération de Draguignan', "Les Sables d'Olonne Agglomération", 'Agglomération de Montpellier', 'Agglomération de Nîmes', 'Agglomération parisienne (hors Paris)', "Agglomération d'Aix-Marseille", 'Auray Quiberon Terre Atlantique', 'Agglomération CIREST', 'Agglomération CASUD', 'Agglomération CINOR', 'Pays de Saint Gilles Croix de Vie', 'Agglomération de Lille', 'Agglomération de Tours', 'Golfe du Morbihan - Vannes Agglomération', 'Agglomération de Teste-de-Buch-Arcachon', 'Pays de Challans', 'Lorient Agglomération', 'Agglomération de Montélimar', 'Agglomération de Clermont-Ferrand', 'Agglomération TCO', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Communes non TLV', 'Agglomération de Bordeaux', "Agglomération d'Arras", 'Agglomération de Toulouse', 'Agglomération de Fréjus', 'Agglomération de Nantes', 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de La Rochelle', 'Agglomération de Nancy', 'Agglomération de Montbéliard'}, (0, 500): {'Agglomération de Vitré', 'Sud Loire - proche Haute-Loire', "Agglomération d'Alençon", 'Agglomération de Brest'}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2019: {(500, 1000): {"Agglomération d'Arles", 'La Roche-sur-Yon Agglomération', 'Agglomération de Grenoble', "Unité urbaine de l'agglomération de Bayonne", 'Eurométropole de Strasbourg', 'Agglomération CIVIS', 'Agglomération de Besançon', 'Agglomération de Sète', "Agglomération d'Ajaccio", 'Agglomération de Valence-Romans', 'Agglomération de Draguignan', "Les Sables d'Olonne Agglomération", 'Agglomération de Montpellier', 'Agglomération de Nîmes', 'Agglomération parisienne (hors Paris)', "Agglomération d'Aix-Marseille", 'Auray Quiberon Terre Atlantique', 'Agglomération CIREST', 'Agglomération CINOR', 'Pays de Saint Gilles Croix de Vie', 'Agglomération de Lille', 'Agglomération de Cap Excellence', 'Agglomération de Tours', 'Agglomération de Bastia', 'Golfe du Morbihan - Vannes Agglomération', 'Pays de Challans', 'Lorient Agglomération', 'Agglomération de Montélimar', 'Agglomération de Clermont-Ferrand', 'Agglomération TCO', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Agglomération de Bordeaux', "Agglomération d'Arras", 'Agglomération de Mâcon', 'Agglomération de Fréjus', 'Agglomération de Toulouse', 'Agglomération de Nantes', 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de Chalon-sur-Saône', 'Agglomération de La Rochelle', 'Agglomération de Nancy', 'Agglomération de Montbéliard'}, (0, 500): {'Agglomération de Vitré', 'Sud Loire - proche Haute-Loire', "Agglomération d'Alençon", 'Agglomération de Brest'}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2020: {(500, 1000): {"Agglomération d'Arles", 'La Roche-sur-Yon Agglomération', 'Agglomération de Grenoble', "Unité urbaine de l'agglomération de Bayonne", 'Eurométropole de Strasbourg', 'Agglomération CIVIS', 'Agglomération de Besançon', 'Agglomération de Saint-Malo', 'Agglomération de Sète', "Agglomération d'Ajaccio", 'Agglomération de Valence-Romans', "Les Sables d'Olonne Agglomération", 'Agglomération de Montpellier', 'Agglomération de Nîmes', 'Agglomération parisienne (hors Paris)', "Agglomération d'Aix-Marseille", 'Auray Quiberon Terre Atlantique', 'Agglomération CIREST', 'Agglomération CASUD', 'Agglomération CINOR', 'Pays de Saint Gilles Croix de Vie', 'Agglomération de Lille', 'Agglomération de Tours', 'Agglomération de Bastia', 'Golfe du Morbihan - Vannes Agglomération', 'Pays de Challans', 'Lorient Agglomération', 'Agglomération de Montélimar', 'Agglomération de Clermont-Ferrand', 'Agglomération TCO', 'Agglomération de Rennes', 'Agglomération de Bordeaux', "Agglomération d'Arras", 'Agglomération de Mâcon', 'Agglomération de Toulouse', 'Agglomération de Nantes', 'Agglomération de Nice-Menton', 'Agglomération de Lyon', 'Agglomération de Chalon-sur-Saône', 'Agglomération de La Rochelle', 'Agglomération de Nancy', 'Agglomération de Montbéliard'}, (0, 500): {'Agglomération de Vitré', "Agglomération d'Alençon", 'Agglomération de Brest'}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2021: {(500, 1000): {'Sud Loire - proche Haute-Loire', "Agglomération d'Arles", 'La Roche-sur-Yon Agglomération', 'Agglomération de Brest', 'Agglomération de Grenoble', 'Eurométropole de Strasbourg', "Unité urbaine de l'agglomération de Bayonne", 'Agglomération CIVIS', 'Agglomération de Besançon', 'Agglomération de La Rochelle', 'Agglomération de Saint-Malo', 'Agglomération de Sète', "Agglomération d'Ajaccio", 'Agglomération de Draguignan', "Les Sables d'Olonne Agglomération", 'Agglomération de Montpellier', 'Agglomération de Nîmes', 'Agglomération parisienne (hors Paris)', "Agglomération d'Aix-Marseille", 'Auray Quiberon Terre Atlantique', 'Agglomération CIREST', 'Agglomération CASUD', 'Agglomération de Vitré', 'Pays de Saint Gilles Croix de Vie', 'Agglomération de Lille', 'Agglomération de Cap Excellence', 'Agglomération de Bastia', 'Golfe du Morbihan - Vannes Agglomération', 'Pays de Challans', 'Lorient Agglomération', 'Agglomération de Clermont-Ferrand', 'Agglomération TCO', 'Agglomération de Rennes', 'Agglomération de Toulon', 'Communes non TLV', 'Agglomération de Bordeaux', "Agglomération d'Arras", 'Agglomération de Mâcon', 'Agglomération de Fréjus', 'Agglomération de Toulouse', 'Agglomération de Nantes', 'Agglomération de Lyon', 'Agglomération de Chalon-sur-Saône', 'Agglomération CINOR', 'Agglomération de Nancy', 'Agglomération de Montbéliard'}, (0, 500): {"Agglomération d'Alençon"}, (1000, 1500): {'Agglomération parisienne', 'Paris intra-muros'}}, 2022: {(500, 1000): {'Golfe du Morbihan - Vannes Agglomération', 'Agglomération de Toulouse', 'Agglomération de Fréjus', 'Lorient Agglomération', 'Agglomération de Nîmes', 'Agglomération de Brest', 'Auray Quiberon Terre Atlantique', 'Communes non TLV', 'Agglomération de Toulon', 'Agglomération de Nancy', 'Agglomération de Draguignan'}, (0, 500): {'Agglomération de Châteauroux'}}}
agglomeration_rent_all_years = ...


In [112]:
agglomeration_rent_per_year_after_2017 = {year: value for year, value in agglomeration_rent_per_year.items() if year>2017}
data_temperature_evolution_rent_range_year = {year: {
    rent_range: pd.concat(dataframes).groupby("Mois").mean().reset_index().sort_values(
        by="Mois", key=lambda serie: [month_dic2[e.split()[0]] for e in serie])
    for rent_range, dataframes in [
        (rent_range, [data_temperature_evolution_agglomeration[(str(year), agglomeration)]
                      for agglomeration in agglomerations
                      if (str(year), agglomeration) in data_temperature_evolution_agglomeration.keys()])
    for rent_range, agglomerations in rent_agglomerations_map.items()
    ]
    if dataframes}
                                         for year, rent_agglomerations_map in agglomeration_rent_per_year.items()}

In [113]:
charts_temperature_evolution_rent_range_year = {
    year: {
        rent_range: px.bar(data,
                           x="Mois",
                           y="TMoy",
                           labels={"Mois":"Month", "TMoy":"Average Temperature (°C)"}).update_layout(title={
            "text":"Evolution of the average temperature in agglomerations<br>with average mensual rent between "
            f"%s € and %s € accross months of {year}"%rent_range,
            "x":0.5,
            "xanchor":"center"})
           for rent_range, data in map_rent_range_data.items()
          }
    for year, map_rent_range_data in data_temperature_evolution_rent_range_year.items()
}

In [114]:
charts_temperature_evolution_rent_range_year[2021][(0, 500)]

In [115]:
charts_temperature_evolution_rent_range_year[2021][(500, 1000)]

In [117]:
charts_temperature_evolution_rent_range_year[2021][(1000, 1500)]

In [116]:
data_temperature_evolution_rent_range = {year: {
    rent_range: pd.concat(dataframes).groupby("Mois").mean().reset_index().sort_values(
        by="Mois", key=lambda serie: [month_dic2[e.split()[0]] for e in serie])
    for rent_range, dataframes in [
        (rent_range, [data_temperature_evolution_agglomeration[(str(year), agglomeration)]
                      for agglomeration in agglomerations
                      if (str(year), agglomeration) in data_temperature_evolution_agglomeration.keys()])
    for rent_range, agglomerations in rent_agglomerations_map.items()
    ]
    if dataframes}
                                         for year, rent_agglomerations_map in agglomeration_rent_per_year.items()}

In [ ]:
data_temperature_evolution_rent_range = {rent_range: }